In [31]:
import spacy
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import paired_cosine_distances

In [7]:
model_name = "en_core_web_md"

In [8]:
def load_model(model_name): 
    try:
        nlp = spacy.load(model_name)
        print(f"Modelo {model_name} cargado exitosamente.")
        return nlp
    except OSError:
        print(f"El modelo {model_name} no está instalado. Descargando e instalando...")
        spacy.cli.download(model_name)
        nlp = spacy.load(model_name)
        return nlp

In [9]:
df = pd.read_parquet('../Datasets_ML/Rest_google.parquet')
df.head()

,name,address,description,category,MISC,coord
0,Roux's Roadhouse 73,"Roux's Roadhouse 73, 784 WI-73, Nekoosa, WI 54457",None,[Restaurant],{'Accessibility': ['Wheelchair accessible entr...,"[44.3069541, -89.8457834]"
1,Crepes n' Tacos Mexican Grill,"Crepes n' Tacos Mexican Grill, suit #5, 5390, ...",None,"[Mexican restaurant, Crêperie, Ice cream shop,...","{'Accessibility': None, 'Amenities': ['Good fo...","[36.1646332, -115.0607804]"
2,The Big Eazy,"The Big Eazy, 2053 N Doctor M.L.K. Jr Dr, Milw...",None,"[Cajun restaurant, Creole restaurant]","{'Accessibility': None, 'Amenities': ['Good fo...","[43.0576581, -87.914643]"
3,Subway,"Subway, 250 E Wolf Run, Mukwonago, WI 53149",Casual counter-serve chain for build-your-own ...,"[Sandwich shop, Caterer, Fast food restaurant,...","{'Accessibility': None, 'Amenities': None, 'At...","[42.8541679, -88.3142471]"
4,Papa Kelsey's,"Papa Kelsey's, 165 E 1400 N, Logan, UT 84341","Chain eatery serving baked subs, classic pizza...",[Pizza restaurant],{'Accessibility': ['Wheelchair accessible entr...,"[41.7580941, -111.8302904]"


In [54]:
len(df)

3223

In [10]:
nlp_en = load_model(model_name)

Modelo en_core_web_md cargado exitosamente.


***

In [11]:
def buscar_palabras_similares(dataframe, columna, lista_busqueda, umbral_similitud=0.7):
    indices = []

    for index,lista in enumerate(dataframe[columna]):
        for palabra in lista:
            similitudes = []
            for palabra_busqueda in lista_busqueda:
                # Calcular la similitud coseno entre los embeddings de las palabras
                embedding_palabra = nlp_en(palabra).vector.reshape(1, -1)
                embedding_palabra_busqueda = nlp_en(palabra_busqueda).vector.reshape(1, -1)
                similitud = cosine_similarity(embedding_palabra, embedding_palabra_busqueda)[0][0]
                similitudes.append(similitud)
            
            # Si la similitud es mayor que el umbral, agregar la palabra
            if max(similitudes) >= umbral_similitud:
                indices.append(index)
                break
    return indices

In [12]:
# Lista de palabras para buscar
palabras_busqueda = ['sushi', 'beer','delivery']

# Buscar palabras iguales o similares en la columna 'palabras'
palabras_similares = buscar_palabras_similares(df, 'category', palabras_busqueda)

print("indices", list(set(palabras_similares)))

indices [513, 1540, 2053, 2055, 1545, 14, 3087, 3089, 1554, 3090, 1556, 22, 23, 1562, 1051, 3098, 2589, 1057, 2593, 2551, 1575, 2599, 3111, 3115, 46, 3120, 3121, 3063, 53, 568, 59, 2620, 1085, 1597, 63, 2621, 578, 3138, 70, 3142, 1101, 3153, 1621, 608, 2146, 3173, 2151, 2663, 1645, 3184, 115, 629, 3191, 2680, 636, 1149, 2172, 2175, 2689, 1155, 3207, 136, 2701, 658, 2194, 148, 150, 671, 160, 2208, 674, 1699, 1704, 170, 1196, 2735, 176, 688, 2743, 2745, 189, 2237, 703, 2753, 2246, 2760, 713, 2761, 204, 206, 2255, 1234, 2771, 1236, 1750, 732, 2781, 734, 735, 2782, 2785, 2788, 743, 744, 746, 2282, 1772, 2284, 2796, 1776, 2289, 2801, 755, 757, 2808, 249, 251, 2813, 2304, 769, 2816, 773, 2823, 776, 2824, 276, 2838, 1303, 2332, 797, 1822, 2337, 2338, 291, 2849, 2350, 1840, 1841, 2352, 307, 820, 1331, 1846, 2874, 317, 831, 320, 839, 840, 846, 1872, 2896, 1362, 2905, 348, 1886, 1888, 2401, 1893, 1384, 2410, 2923, 876, 368, 1907, 2933, 1913, 381, 391, 2441, 1418, 2442, 2956, 911, 1938, 410, 2970

In [14]:
len(list(set(palabras_similares)))

217

***

Version de la funcion optimizada

Es esta version se busco reducir la cantidad de bucles for para mejorar el desempeño de la funcion.

In [15]:
def buscar_palabras_similares(dataframe, columna, lista_busqueda, umbral_similitud=0.7):
    indices = []

    # Calcular embeddings para las palabras de la lista de búsqueda
    embeddings_busqueda = np.array([nlp_en(palabra).vector for palabra in lista_busqueda])

    for index, palabras_lista in enumerate(dataframe[columna]):
        # Calcular embeddings para las palabras de la lista actual
        embeddings_lista = np.array([nlp_en(palabra).vector for palabra in palabras_lista])
        
        # Calcular la similitud coseno entre los embeddings de las palabras de la lista actual y las de búsqueda
        similitudes = cosine_similarity(embeddings_lista, embeddings_busqueda).max(axis=1)
        
        # Si la similitud es mayor que el umbral, agregar el índice
        if np.any(similitudes >= umbral_similitud):
            indices.append(index)

    return indices

In [16]:
# Lista de palabras para buscar
palabras_busqueda = ['sushi', 'beer','delivery']

# Buscar palabras iguales o similares en la columna 'palabras'
palabras_similares = buscar_palabras_similares(df, 'category', palabras_busqueda)

print("indices", list(set(palabras_similares)))

indices [513, 1540, 2053, 2055, 1545, 14, 3087, 3089, 1554, 3090, 1556, 22, 23, 1562, 1051, 3098, 2589, 1057, 2593, 2551, 1575, 2599, 3111, 3115, 46, 3120, 3121, 3063, 53, 568, 59, 2620, 1085, 1597, 63, 2621, 578, 3138, 70, 3142, 1101, 3153, 1621, 608, 2146, 3173, 2151, 2663, 1645, 3184, 115, 629, 3191, 2680, 636, 1149, 2172, 2175, 2689, 1155, 3207, 136, 2701, 658, 2194, 148, 150, 671, 160, 2208, 674, 1699, 1704, 170, 1196, 2735, 176, 688, 2743, 2745, 189, 2237, 703, 2753, 2246, 2760, 713, 2761, 204, 206, 2255, 1234, 2771, 1236, 1750, 732, 2781, 734, 735, 2782, 2785, 2788, 743, 744, 746, 2282, 1772, 2284, 2796, 1776, 2289, 2801, 755, 757, 2808, 249, 251, 2813, 2304, 769, 2816, 773, 2823, 776, 2824, 276, 2838, 1303, 2332, 797, 1822, 2337, 2338, 291, 2849, 2350, 1840, 1841, 2352, 307, 820, 1331, 1846, 2874, 317, 831, 320, 839, 840, 846, 1872, 2896, 1362, 2905, 348, 1886, 1888, 2401, 1893, 1384, 2410, 2923, 876, 368, 1907, 2933, 1913, 381, 391, 2441, 1418, 2442, 2956, 911, 1938, 410, 2970

In [17]:
len(list(set(palabras_similares)))

217

***

In [32]:
model_name = "bert-base-uncased"

In [33]:
def load_bert_model(model_name):
    tokenizer = BertTokenizer.from_pretrained(model_name)
    model = BertModel.from_pretrained(model_name)

    # Aseguramos que el modelo se cargue en la GPU si está disponible
    if torch.cuda.is_available():
        model.to('cuda')

    print(f"Modelo {model_name} cargado exitosamente.")
    return model, tokenizer

In [34]:
bert_model, bert_tokenizer = load_bert_model(model_name)

Modelo bert-base-uncased cargado exitosamente.


In [35]:
def process_text(text, model, tokenizer):
    # Tokenizar y convertir a IDs de tokens
    inputs = tokenizer(text, return_tensors="pt")
    
    # Asegurarse de que los cálculos se realicen en la GPU si está disponible
    if torch.cuda.is_available():
        inputs = {key: val.to('cuda') for key, val in inputs.items()}
    
    # Obtener la salida de BERT (representación oculta)
    with torch.no_grad():
        outputs = model(**inputs)
    
    # La salida contiene embeddings de palabras y otros detalles, aquí estamos obteniendo solo los embeddings
    embeddings = outputs.last_hidden_state
    
    # Si las operaciones fueron realizadas en la GPU, es necesario mover los embeddings a la CPU para su uso
    embeddings = embeddings.cpu().numpy()
    
    # Colapsar las dimensiones batch_size y sequence_length
    embeddings_collapsed = embeddings.mean(axis=1)  # Puedes usar otras estrategias de colapso si es necesario
    
    return embeddings_collapsed

In [52]:
def buscar_palabras_similares(dataframe, columna, lista_busqueda, umbral_similitud=0.75):
    indices = []
    palabras = []

    # Calcular embeddings para las palabras de la lista de búsqueda
    embeddings_busqueda = process_text(" ".join(lista_busqueda), bert_model, bert_tokenizer)

    for index, palabras_lista in enumerate(dataframe[columna]):
        # Calcular embeddings para las palabras de la lista actual
        embeddings_lista = process_text(" ".join(palabras_lista), bert_model, bert_tokenizer)
        
        # Calcular la similitud coseno con normalización por la norma euclidiana
        similitudes = 1 - paired_cosine_distances(embeddings_lista, embeddings_busqueda)
        
        # Si la similitud es mayor que el umbral, agregar el índice
        if np.any(similitudes >= umbral_similitud):
            indices.append(index)
            palabras.append(palabras_lista)
    return indices,palabras

In [53]:
# Lista de palabras para buscar
palabras_busqueda = ['sushi', 'beer','delivery']

# Buscar palabras iguales o similares en la columna 'palabras'
indices,palabras_similares = buscar_palabras_similares(df, 'category', palabras_busqueda)

print("indices", list(set(indices)))
print("indices", palabras_similares)

indices [2049, 2051, 4, 2053, 2058, 15, 16, 2065, 19, 2068, 2072, 28, 29, 30, 32, 2083, 2085, 2086, 2087, 41, 42, 45, 2094, 47, 2095, 2096, 2100, 53, 66, 69, 2118, 2120, 2121, 74, 75, 76, 2129, 82, 83, 2133, 86, 88, 91, 2140, 93, 2143, 96, 97, 2144, 99, 100, 2145, 102, 2147, 2151, 112, 113, 114, 115, 2168, 122, 2171, 2174, 2175, 2177, 2178, 2180, 134, 135, 136, 2182, 2187, 2188, 141, 142, 2191, 144, 2197, 151, 2200, 153, 2204, 2207, 162, 164, 2213, 166, 2216, 2217, 2219, 2221, 2222, 175, 2224, 179, 2228, 181, 2231, 184, 185, 2235, 189, 194, 195, 2246, 2248, 2249, 203, 2253, 206, 2254, 2256, 209, 210, 215, 217, 2265, 2266, 220, 2268, 2269, 2275, 228, 229, 231, 2280, 233, 2281, 238, 2288, 2292, 251, 252, 253, 254, 2302, 256, 2303, 258, 2307, 2309, 2310, 265, 266, 267, 2314, 269, 2318, 271, 2320, 2322, 277, 2325, 279, 2328, 283, 2331, 2333, 286, 287, 2334, 2338, 2339, 292, 293, 296, 300, 301, 2349, 2350, 2353, 306, 307, 2354, 2355, 2357, 311, 2361, 2367, 322, 2373, 326, 327, 328, 2374, 23

In [44]:
len(list(set(indices)))

39